In [33]:
import pyspark as ps
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS
import pyspark.sql.functions as F
from pyspark.sql.types import StructType, StructField
from pyspark.sql.types import IntegerType, StringType, FloatType, DateType, TimestampType
import pyspark.sql.functions as F
from pyspark.mllib.evaluation import RegressionMetrics
from pyspark.sql.functions import isnan, when, count, col
# import matplotlib.pyplot as plt
# from MatrixFactorizationModel

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

No module named 'plotly'
Traceback (most recent call last):
ModuleNotFoundError: No module named 'plotly'



# First section: descriptions of datasets used to train model. Winner: final_anime_2.csv

In [62]:
# tiny test set to try
user_test = spark.read.csv('s3://myanimelist-recommender/test_user_profile.csv', header = True)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
user_test = user_test.withColumn("anime_id", user_test.anime_id.cast(IntegerType()))
user_test = user_test.withColumn("my_score", user_test.my_score.cast(IntegerType()))
user_test= user_test.withColumn("user_id", user_test.user_id.cast(IntegerType()))
user_test = user_test.withColumn("stats_mean_score", user_test.stats_mean_score.cast(FloatType()))
# data = data.withColumn("my_watched_episodes", data.my_score.cast(IntegerType()))
# data = data.withColumn("my_status", data.my_score.cast(IntegerType()))
user_test

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

name 'user_test' is not defined
Traceback (most recent call last):
NameError: name 'user_test' is not defined



In [22]:
# This is data iteration one: it is compprised of 21994566 rows × 8 columns (blank, anime_id, username, user_id, my_score, my_watched_episodes, my_status)
# my status 1 - 3 only were used because they indicate 1: watching 2: completed 3: on hold 4: dropped 6: plan to watch
# so only 1-3  were what i considered really relevant, as droppers probably dont have fully formed opinions

# my_watched episodes was another metric i used to choose who to drop. anybody who had watched less than
# 3 episodes of a show likely did not have what i would consider a fully formed opinion so they were dropped.

# Furthermore, a large number of these made up the 0 ratings which were in overabundance
data = spark.read.csv('s3://myanimelist-recommender/final_anime.csv', header = True)
data.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---+--------+--------+--------+-------+----------------+-------------------+---------+
|_c0|anime_id|username|my_score|user_id|stats_mean_score|my_watched_episodes|my_status|
+---+--------+--------+--------+-------+----------------+-------------------+---------+
|  0|      21|karthiga|       9|2255153|            7.43|                586|        1|
|  1|      59|karthiga|       7|2255153|            7.43|                 26|        2|
|  2|      74|karthiga|       7|2255153|            7.43|                 26|        2|
|  3|     120|karthiga|       7|2255153|            7.43|                 26|        2|
|  4|     178|karthiga|       7|2255153|            7.43|                 26|        2|
|  5|     210|karthiga|       7|2255153|            7.43|                161|        2|
|  6|     232|karthiga|       6|2255153|            7.43|                 70|        2|
|  7|     233|karthiga|       6|2255153|            7.43|                 78|        2|
|  8|     249|karthiga|       8|

In [5]:
data.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

15546712

In [22]:
# data iteration 2: it is one with similar conditions to the above, with the addition of
# filtering out any user or anime with less than 5 reviews. This will hopefully increase the 
# overall effectiveness of the model despite removing data
data = spark.read.csv('s3://myanimelist-recommender/final_anime_2.csv', header = True)
data.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

15546712

In [23]:
data = data.withColumn("anime_id", data.anime_id.cast(IntegerType()))
data = data.withColumn("user_id", data.user_id.cast(IntegerType()))
data = data.withColumn("my_score", data.my_score.cast(IntegerType()))
data = data.withColumn("stats_mean_score", data.stats_mean_score.cast(FloatType()))
# data = data.withColumn("my_watched_episodes", data.my_score.cast(IntegerType()))
# data = data.withColumn("my_status", data.my_score.cast(IntegerType()))

# data = data.drop('_c0')
data.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---+--------+--------+-------+----------------+
|_c0|anime_id|my_score|user_id|stats_mean_score|
+---+--------+--------+-------+----------------+
|  0|      21|       9|2255153|            7.43|
|  1|      59|       7|2255153|            7.43|
|  2|      74|       7|2255153|            7.43|
|  3|     120|       7|2255153|            7.43|
|  4|     178|       7|2255153|            7.43|
|  5|     210|       7|2255153|            7.43|
|  6|     232|       6|2255153|            7.43|
|  7|     233|       6|2255153|            7.43|
|  8|     249|       8|2255153|            7.43|
|  9|     269|      10|2255153|            7.43|
| 10|     721|       7|2255153|            7.43|
| 11|     853|       7|2255153|            7.43|
| 12|     857|      10|2255153|            7.43|
| 13|     957|       8|2255153|            7.43|
| 14|     966|      10|2255153|            7.43|
| 15|    1557|       8|2255153|            7.43|
| 16|    1571|       8|2255153|            7.43|
| 17|    1579|      

In [2]:
# This is data iteration three (final_anime_3.csv). In this, users and anime have been further filtered
# only users who have rated >6 and anime that have been rated >6 times are included in this dataset. 
# otherwise it is the same as iteration 2. 
data = spark.read.csv('s3://myanimelist-recommender/anime_final_3.csv', header = True)
data.count()


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

15022090

In [13]:
data = data.withColumn("anime_id", data.anime_id.cast(IntegerType()))
data = data.withColumn("user_id", data.user_id.cast(IntegerType()))
data = data.withColumn("my_score", data.my_score.cast(IntegerType()))
data = data.withColumn("stats_mean_score", data.stats_mean_score.cast(FloatType()))
# data = data.withColumn("my_watched_episodes", data.my_score.cast(IntegerType()))
# data = data.withColumn("my_status", data.my_score.cast(IntegerType()))

# data = data.drop('_c0')
# data = data.drop('my_watched_episodes')
data = data.drop( 'my_status')
data.show()


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---+--------+--------+--------+-------+----------------+
|_c0|anime_id|username|my_score|user_id|stats_mean_score|
+---+--------+--------+--------+-------+----------------+
|  0|      21|karthiga|       9|2255153|            7.43|
|  1|      59|karthiga|       7|2255153|            7.43|
|  2|      74|karthiga|       7|2255153|            7.43|
|  3|     120|karthiga|       7|2255153|            7.43|
|  4|     178|karthiga|       7|2255153|            7.43|
|  5|     210|karthiga|       7|2255153|            7.43|
|  6|     232|karthiga|       6|2255153|            7.43|
|  7|     233|karthiga|       6|2255153|            7.43|
|  8|     249|karthiga|       8|2255153|            7.43|
|  9|     269|karthiga|      10|2255153|            7.43|
| 10|     721|karthiga|       7|2255153|            7.43|
| 11|     853|karthiga|       7|2255153|            7.43|
| 12|     857|karthiga|      10|2255153|            7.43|
| 13|     957|karthiga|       8|2255153|            7.43|
| 14|     966|

In [18]:
# anime5 - copy of anime2 except
# keepies = hope.groupby('anime_id').count()[hope.groupby('anime_id').count()['user_id']>69].index
# hope = hope[hope['anime_id'].isin(keepies)]
# keepies3 = ratings_user.sort_values(by = 'anime_id', ascending = False)[ratings_user.sort_values(by = 'anime_id',ascending = False)['anime_id'] >13].index
# hope = hope[hope['anime_id'].isin(keepies3)]
data = spark.read.csv('s3://myanimelist-recommender/anime_final_5.csv', header = True)
data = data.withColumn("anime_id", data.anime_id.cast(IntegerType()))
data = data.withColumn("user_id", data.user_id.cast(IntegerType()))
data = data.withColumn("my_score", data.my_score.cast(IntegerType()))
data = data.withColumn("stats_mean_score", data.stats_mean_score.cast(FloatType()))


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

# Train-Test-Split and Model

In [41]:
training, testing = data.randomSplit([0.8, 0.2])

# training = training.drop('_c0')
# testing = testing.drop('_c0')
training.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

12439797

In [13]:
# best split i made, read it in to test things 
# train = spark.read.csv('s3://myanimelist-recommender/final_anime_2_test.csv/part-00000-b5e9b05f-0841-470b-920b-5403990a8218-c000.csv', header = True)
# train.count()

# test =spark.read.csv('s3://myanimelist-recommender/test1.csv', header = True) 

testing

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[_c0: string, anime_id: int, my_score: int, user_id: int, stats_mean_score: float]

In [41]:
# for read in train, test
train= train.withColumn("anime_id", train.anime_id.cast(IntegerType()))
train = train.withColumn("user_id", train.user_id.cast(IntegerType()))
train= train.withColumn("my_score", train.my_score.cast(IntegerType()))
train= train.withColumn("stats_mean_score", train.stats_mean_score.cast(FloatType()))

test = test.withColumn("anime_id", test.anime_id.cast(IntegerType()))
test = test.withColumn("user_id", test.user_id.cast(IntegerType()))
test = test.withColumn("my_score", test.my_score.cast(IntegerType()))
test = test.withColumn("stats_mean_score", test.stats_mean_score.cast(FloatType()))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [39]:
# See how similar the my_score is to test set my_score


# display(data.select('my_score', 'user_id').groupby('user_id').agg(avg('my_Score')))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+-------+
|my_score|  count|
+--------+-------+
|       1|  40445|
|       6|1362717|
|       3| 114095|
|       5| 624838|
|       9|2578781|
|       4| 256723|
|       8|3513889|
|       7|2903623|
|      10|1837597|
|       2|  61110|
|       0|2252894|
+--------+-------+

In [44]:
"""
Here i print out the spread of the split. I dont want the testing group and the training groups contents 
in terms of makeup to be too dissimilar or the recommender will fail to work properly, so this allows
me to show it is indeed working
"""
# print("users in training: {}".format(training.select('user_id').distinct().count()))
print("users in testing: {}".format(testing.select('user_id').distinct().count()))

print("users in both: {}".format(training.select('user_id').distinct()\
                                         .join(testing.select('user_id').distinct(),
                                               'user_id', 'inner')\
                                         .count()))

# print("anime in training: {}".format(training.select('anime_id').distinct().count()))
print("anime in testing: {}".format(testing.select('anime_id').distinct().count()))

print("anime in both: {}".format(training.select('anime_id').distinct()\
                                         .join(testing.select('anime_id').distinct(),
                                               'anime_id', 'inner')\
                                         .count()))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

users in testing: 104548
users in both: 104522
anime in testing: 3547
anime in both: 3531

In [45]:
class AnimeRecommender_loop1():
    """Loop 1 recommender
    Uses ALS out of the box"""
    def fit(self, train_df):
        self.als = ALS(rank=200,
              maxIter=18,
              regParam=0.1,
              userCol="user_id",
              itemCol="anime_id",
              ratingCol="my_score")

        self.loop1_model = self.als.fit(train_df)

    def transform(self, test_df):
        return(self.loop1_model.transform(test_df))


class AnimeRecommender_loop2():
    """Loop 2 recommender 
    Uses average per anime to fill up NaNs left by ALS"""
    def fit(self, training_df):
        self.avg_ratings = training_df.select('anime_id','my_score', 'stats_mean_score')\
                                      .groupBy('anime_id')\
                                      .agg(F.avg('my_score'))\
                                      .withColumnRenamed('avg(my_score)','avg_rating')

    def transform(self, requests_df):
        return(requests_df.join(self.avg_ratings, 'anime_id', 'left')\
                          .withColumnRenamed('avg_rating','prediction'))


class AnimeRecommender(ALS):
    """Aggregate the results from loop1 and loop2 """
    def __init__(self):
        self.spark = ps.sql.SparkSession.builder \
                    .master("local[4]") \
                    .appName("df lecture") \
                    .getOrCreate()
        self.sc = self.spark.sparkContext  # for the pre-2.0 sparkContext
        self.ar1 = AnimeRecommender_loop1()
        self.ar2 = AnimeRecommender_loop2()


    def fit(self, ratings):
        """
        Trains the recommender on a given set of ratings.
        Parameters
        ----------
        ratings : pandas dataframe, shape = (n_ratings, 4)
                  with columns 'user', 'movie', 'rating', 'timestamp'
        Returns
        -------
        self : object
            Returns self.
        """
        

        self.training = ratings
        self.training.persist()

        self.ar1.fit(self.training)
        self.ar2.fit(self.training)

        return(self)


    def transform(self, requests):
        """
        Predicts the ratings for a given set of requests.
        Then fills NaN values with avg. rating of the anime
        Parameters
        ----------
        requests : pandas dataframe, shape = (n_ratings, 2)
                  with columns 'user', 'movie'
        Returns
        -------
        dataframe : a *pandas* dataframe with columns 'user', 'movie', 'rating'
                    column 'rating' containing the predicted rating
        """

        self.requests = requests

        # GET FIRST ROUND PRED
        pred_loop1 = self.ar1.transform(self.requests).withColumnRenamed('prediction','prediction_loop1')
        # option to show        
        # print(pred_loop1.show())

        # GET AVERAGES OF VALS TO FILLNA FOR MISSING VALUES AFTER PRED_LOOP1
        pred_loop2 = self.ar2.transform(pred_loop1).withColumnRenamed('prediction','prediction_loop2')
        #option to show
        #  print(pred_loop2.show())

        # FILLNA USING PRED_LOOP2. OPTION TO FILLNA WITH USER AVERAGE IF NEEDED

        results_loop2 = pred_loop2.withColumn('prediction',F.when(F.isnan('prediction_loop1'),F.col('prediction_loop2')).otherwise(F.col('prediction_loop1')))
        
        # if you want to see how many Nan's remain, free up 3 lines below

        # print(results_loop2.count())
        # print(results_loop2.dropna(subset=('prediction_loop1')).count())
        # results_loop2 = pred_loop2.withColumn('prediction',F.when(F.isnan('prediction_loop1'),F.col('prediction_loop2')).otherwise(F.col('prediction_loop1')))

        predictions = results_loop2.select('user_id', 'anime_id', 'my_score', 'prediction')\
                                   .withColumnRenamed('prediction','rating')
        
        
        # see how many nans lost here and drop them for testing

        # print(predictions.count())
        preds = predictions.dropna(subset=('rating'))
        # print(preds.count())
        return preds
    
    def evaluate(self, predictions):
        evaluator = RegressionEvaluator(metricName="rmse", labelCol="my_score",predictionCol="rating")
        
        rmse = evaluator.evaluate(predictions)
        return print(f'the rmse is {rmse}')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

# Instantiation and training

In [46]:
ar = AnimeRecommender()
x = ar.fit(training)
preds = ar.transform(testing)


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+--------+--------+-------+----------------+----------------+-----------------+
|anime_id|     _c0|my_score|user_id|stats_mean_score|prediction_loop1| prediction_loop2|
+--------+--------+--------+-------+----------------+----------------+-----------------+
|     148| 1999863|       6|  91084|            7.16|        6.449814|5.840686274509804|
|     148|19000468|       9| 120976|            7.13|        7.184175|5.840686274509804|
|     148|15170943|       8|  19701|            7.46|        4.001227|5.840686274509804|
|     148| 5340083|      10| 178467|            7.81|        6.204935|5.840686274509804|
|     148|11055702|      10|3535317|            10.0|        9.088889|5.840686274509804|
|     148|14856483|       0|   1005|             0.0|             0.0|5.840686274509804|
|     148| 2662989|       7| 312632|            6.11|       6.6216993|5.840686274509804|
|     148|16290231|      10| 316289|            9.31|        8.656877|5.840686274509804|
|     148| 8095348|  

In [16]:
# count of dropped nans between loop1 and end =  75


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

75

In [47]:
# cvgrid is not working, so ill do tuning by hand in the next cells
# evaluator uses root mean square error to see how distant predictions are from scores
evaluator = RegressionEvaluator(metricName="rmse", labelCol="my_score",predictionCol="rating")
        
rmse = evaluator.evaluate(preds)


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [48]:
# for anime_final_2: see comments at top
# rmse = 2.1254230685892352 rank = 100, maxIter = 10, reg param = .1
# rmse = 2.1058600851187363 rank = 100, maxiter = 15, regparam =.1
# rmse = 2.0987042601431383 rank = 150, maxiter = 15, regparam = .1
# rmse = 1.6232912469115457, rank 200, maxiter 17, regparam .1 <----- huge win, saved these train splits
# rmse = 1.6143999936595663, rank 220, maxiter 18, regparam = .1 <------ bigger win, used a different split^
# rmse = 2.102241385301613 rank 220, maxiter 18, regoaram .1, ? fluke check on what i thought were same train split as huge win. may have made error recombining in eda file
# rmse = 2.0934816014194326, rank 200, maxiter 18, reparam.1, < 25-75 train test split
# rmse = 2.1059523655991232 rank 200, maxiter 18 - another attempt, cant recreate early success for unknown reason
# rmse = 2.084936624781888 rank 200, maxiter 18 - another attempt,
# rmse = 2.083972607916511, rank 200, maxiter 18 - another attempt,
# rmse = 2.093279077573417,rank 200, maxiter 18 - another attempt,
# rmse = 2.085884408015234. rank 200, maxiter 18 - another attempt

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

2.085884408015234

## losing sets

In [81]:
# for anime_final: see comments above
# rmse = 1.8450456842750507 on initial pass of data set, rank = default, maxIter = default, my_status <3, minimum watched >0
# ^unrecreatable for some reason
# rmse = 2.1858678093919712 on another pass, rank = 100, maxIter = 10, reg param = .1
# rmse = 2.175129033735969 on rank = 250, maxIter = 10, regparam = .1
# rmse =  2.1712464062761363 on rank 100, maxIter = 15, regparam = .1
# rmse = 2.6971824770818507 on rank 100, maxIter = 10, reparam = .01
# rmse = 2.1791087286348882, rank 120, maxiter = 11, regparam =.1

rmse

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

2.1800007704960653

In [24]:
# for final_anime_3: see comments at top
# rmse =1.9578590462477  rank 200, maxiter 17, regparam .1
# rmse = 2.0000903889660986, rank 100, maxiter 10, regparam .1
rmse

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

2.0000903889660986

## Saving things to s3

In [15]:
training=training.repartition(1)
training.write.save("s3://myanimelist-recommender/final_anime_2_test.csv", format='csv', header=True)
testing = testing.repartition(1)
testing.write.save("s3://myanimelist-recommender/final_anime_2_train.csv", format='csv', header=True)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [21]:
preds=preds.repartition(1)
preds.write.save("s3://myanimelist-recommender/predictions_example_2.csv", format='csv', header=True)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [43]:
testing = testing.repartition(1)
testing.write.save('s3://myanimelist-recommender/test_set.csv')

training = training.repartition(1)
training.write.save('s3://myanimelist-recommender/train_set.csv')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
model.save([spark_context], [file_path])
